# Sandbox pour développer et tester + facilement

L'idée c'est de tester ici (c'est quand même beaucoup plus simple que de manier le REPL Julia ou de run un fichier entier à chaque fois) pour accélérer le dévelopement :-)

In [ ]:
using JuMP, Gurobi

include(joinpath("src", "read.jl"))

In [ ]:
n, d, f, Amin, Nr, R, regions, coords, D = readInstance(joinpath("data", "instance_6_1.txt"))

neighb = Vector{Set{Int64}}(undef, n)
arcs = Set{Tuple{Int64, Int64}}()

for i in 1:n
    neighb[i] = Set{Int64}()
    for j in 1:n
        if D[i, j] <= R && j != i
            push!(neighb[i], j)
            push!(arcs, (i, j))
        end
    end
end

println("nb sommets = ", n)
println("depart = ", d)
println("fin = ", f)
println("min nb aerodromes a visiter = ", Amin)
println("nb regions = ", Nr)
println("R = ", R)


In [ ]:
arcs

In [ ]:
neighb

In [ ]:
struct GraphInfo
    neighb::Vector{Set{Int64}}
    arc_values::Any
end

mutable struct TarjanState
    fifo::Vector{Int64}
    is_in_fifo::Vector{Bool}
    first_seen_time::Vector{Int64}
    lowest_accessible::Int64
    time::Int64
    components::Vector{Vector{Int64}}
end

function rec_tarjan(node::Int64, graph_info::GraphInfo, state::TarjanState)
    state.first_seen_time[node] = state.time
    state.lowest_accessible[node] = state.time
    state.time += 1
    push!(state.fifo, node)
    state.is_in_fifo[node] = true
    for v in graph_info.neighb[node]
        if state.first_seen_time[v] == -1 && graph_info.arc_values[(node, v)] > 0
            rec_tarjan(node=v, graph_info=graph_info, state=state)
        end
        if state.is_in_fifo[v]
            state.lowest_accessible[node] = min(state.lowest_accessible[node], state.lowest_accessible[v])
        end
    end
    if state.lowest_accessible[node] == state.first_seen_time[node]
        new_comp = Vector{Int64}(undef, 0)
        to_add = -1
        while to_add != node
            to_add = pop!(state.fifo)
            push!(new_comp,  to_add)
            state.is_in_fifo[to_add] = false
        end
        push!(new_comp, to_add)
    end
end

function strongConnectedComps(neighb::Vector{Set{Int64}}, arc_values)::Vector{Vector{Int64}}

end